In [60]:
#1. Create a table showing constituent (stocks) risk analysis in the equal-weight portfolio analysis as of the
#current date.
#a. Column 1 – Ticker [Complete]
#b. Column 2 – Portfolio Weight (equally weighted) [Complete]
#c. Column 3 – Annualized Volatility (using trailing 3-months)
#d. Column 4 – Beta against SPY (using trailing 12-months)
#e. Column 5 – Beta against IWM (using trailing 12-months)
#f. Column 6 – Beta against DIA (using trailing 12-months
#g. Column 7 – Average Weekly Drawdown (52-week Low minus 52-week High) / 52-week High
#h. Column 8 – Maximum Weekly Drawdown (52-week Low minus 52-week High) / 52-week High
#i. Column 9 – Total Return (using trailing 10-years)
#j. Column 10 – Annualized Total Return (using trailing 10-years)

import pandas as pd
import yfinance as yf
import numpy as np

In [61]:
# Tickers on the NYSE traded assets, 7 stocks and 3 ETFS
tickers = ['NVDA', 'TSLA', 'GME','AMD','MSFT','META','WMT', 'SPY', 'IWM', 'DIA']

In [62]:
# Downloading the historical data, 10 years is a safe number
start_date = '2012-11-23'
end_date = '2022-11-23'
data = yf.download(tickers, start = start_date, end = end_date)['Adj Close']

# Need to compare the 7 traded assets to the ETFS, so the ETFS are benchmarks
benchmarks = yf.download(['SPY', 'DIA', 'IWM'], start = start_date, end = end_date)['Adj Close']


[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  3 of 3 completed


In [63]:
benchmarks

Ticker,DIA,IWM,SPY
Date,,,
2012-11-23 00:00:00+00:00,100.810196,68.453934,113.751717
2012-11-26 00:00:00+00:00,100.523048,68.675110,113.510269
2012-11-27 00:00:00+00:00,99.871124,68.590034,112.930878
2012-11-28 00:00:00+00:00,100.616158,69.168495,113.840279
2012-11-29 00:00:00+00:00,101.004234,69.959641,114.371376
...,...,...,...
2022-11-16 00:00:00+00:00,322.862427,179.751617,384.126862
2022-11-17 00:00:00+00:00,322.891205,178.083359,382.951508
2022-11-18 00:00:00+00:00,324.769318,179.234573,384.690277


In [64]:
# Portfolio Weight: A percentage of an investment portfolio that a specific asset or holding represents
# Formula = [(Stock's Value) / (Total Portfolio Value)] * 100. In this case they all weigh the same amount.
portfolioWeight = np.ones(len(tickers)) / len(tickers)*100

# Create DataFrame with tickers as index and portfolio weights as a column
df = pd.DataFrame({'Weight': portfolioWeight}, index=tickers)
print(df)


      Weight
NVDA    10.0
TSLA    10.0
GME     10.0
AMD     10.0
MSFT    10.0
META    10.0
WMT     10.0
SPY     10.0
IWM     10.0
DIA     10.0


Since there are 10 assets and there isn't a specific money distribution, they all weigh the same in the portfolio as shown in the dataframe.

In [67]:
# Annualized Volatilty is used to indicate how much the value of an investment is likely to fluctuate
# Formula = Standard Deviation x sqrt(252) where 252 is the annual that excludes weekends and holidays

data[tickers]


Ticker,NVDA,TSLA,GME,AMD,MSFT,META,WMT,SPY,IWM,DIA
Date,,,,,,,,,,
2012-11-23 00:00:00+00:00,0.274612,2.142000,4.685037,1.950000,22.516846,23.927956,18.180353,113.751717,68.453934,100.810196
2012-11-26 00:00:00+00:00,0.279458,2.151333,4.685037,1.870000,22.264851,25.862133,18.105249,113.510269,68.675110,100.523048
2012-11-27 00:00:00+00:00,0.280612,2.143333,4.660581,1.880000,22.012859,26.071501,17.999069,112.930878,68.590034,99.871124
2012-11-28 00:00:00+00:00,0.282919,2.215333,4.618657,1.960000,22.240459,26.280872,18.273588,113.840279,69.168495,100.616158
2012-11-29 00:00:00+00:00,0.277843,2.246000,4.578479,2.040000,21.907183,27.237991,18.343513,114.371376,69.959641,101.004234
...,...,...,...,...,...,...,...,...,...,...
2022-11-16 00:00:00+00:00,15.895466,186.919998,27.139999,72.699997,238.303055,112.890099,48.104252,384.126862,179.751617,322.862427
2022-11-17 00:00:00+00:00,15.662678,183.169998,27.760000,73.900002,238.253738,111.115448,47.942284,382.951508,178.083359,322.891205
2022-11-18 00:00:00+00:00,15.394923,180.190002,27.600000,73.570000,237.800247,111.713646,48.664658,384.690277,179.234573,324.769318


In [69]:
benchmarks

Ticker,DIA,IWM,SPY
Date,,,
2012-11-23 00:00:00+00:00,100.810196,68.453934,113.751717
2012-11-26 00:00:00+00:00,100.523048,68.675110,113.510269
2012-11-27 00:00:00+00:00,99.871124,68.590034,112.930878
2012-11-28 00:00:00+00:00,100.616158,69.168495,113.840279
2012-11-29 00:00:00+00:00,101.004234,69.959641,114.371376
...,...,...,...
2022-11-16 00:00:00+00:00,322.862427,179.751617,384.126862
2022-11-17 00:00:00+00:00,322.891205,178.083359,382.951508
2022-11-18 00:00:00+00:00,324.769318,179.234573,384.690277
